# NYCbuswatcher API Demo (against api.buswatcher.org)

In [1]:
# !pip install requests pandas geopandas ipyleaflet ipywidgets matplotlib python-dateutil pydeck

In [2]:
import requests
import pandas as pd
import geopandas as gpd

%matplotlib inline

Unable to revert mtime: /Library/Fonts
Fontconfig warning: ignoring UTF-8: not a valid region tag


In [3]:
# prevent cell wrapping in dataframe tables

In [4]:
%%html
<style>
.dataframe td {
    white-space: nowrap;
}
</style>

## How to use the API:

A Shipment is a JSON file showing all observations of a single route for a single hour. 
There are several endpoints that let you:
- get a list of available Shipments to retrive
- retrieve the Shipments as JSON or GeoJOSN.

In [5]:
api_url = 'http://api.buswatcher.org/api/v2'

## Get a list of all the Shipments in history for a specific route

#### ENDPOINT: List All Shipments In History For Route: `/api/v2/nyc/{route}` 

In [6]:
route='M15'

In [7]:
shipments_for_route_url = f'{api_url}/nyc/{route}'
shipments_for_route_url

'http://api.buswatcher.org/api/v2/nyc/M15'

In [8]:
# fetch the index of shipments for this route
shipment_list = requests.get(shipments_for_route_url).json()

In [9]:
# inspect the response
list(shipment_list.keys())

['route', 'shipments']

In [10]:
# verify route
shipment_list['route']

'M15'

In [11]:
# how many shipment pointers in all
len(shipment_list['shipments'])

1266

In [12]:
# look at first record (shipments is a list of dicts, one for each shipment in the Data Store)
shipment_list['shipments'][0]

{'route': 'M15',
 'year': 2021,
 'month': 6,
 'day': 30,
 'hour': 23,
 'url': 'http://api.buswatcher.org:80/api/v2/nyc/2021/6/30/23/M15/buses'}

In [13]:
# and the last one
shipment_list['shipments'][-1]

{'route': 'M15',
 'year': 2021,
 'month': 9,
 'day': 3,
 'hour': 19,
 'url': 'http://api.buswatcher.org:80/api/v2/nyc/2021/9/3/19/M15/buses'}

#### grab one shipment... 

In [ ]:
url = shipment_list['shipments'][0]['url']
shipment = requests.get(url).json()

#### look inside it... 

In [ ]:
# how many buses?
len(shipment['buses'])

In [ ]:
# look at one record
shipment['buses'][0]

In [ ]:
# we can also get the shipment as geojson, jsut add /geojson to the end of the endpoint

url = shipment_list['shipments'][0]['url'] + '/geojson'
gj_shipment = requests.get(url).json()

In [ ]:
# verify same # of buses
len(gj_shipment['features'])

In [ ]:
# and check out the geojson (hint: you can paste this into geojson.io to make maps instantly)
gj_shipment['features'][0]

#### ...and map it

In [ ]:
from ipywidgets import Layout
from ipyleaflet import (
    Map, basemaps, basemap_to_tiles,
    Circle, Marker, Rectangle, LayerGroup
)

# init the data layer
buses_gdf = gpd.GeoDataFrame.from_features(gj_shipment['features'])

# init the map
defaultLayout=Layout(width='960px', height='540px')
center = (40.7128, -74.0060) #reverse?
zoom = 12


toner = basemap_to_tiles(basemaps.Stamen.Toner)
m = Map(layers=(toner, ), center=center, zoom=zoom, layout=defaultLayout)

# Create layer group
layer_group = LayerGroup()


for index, row in buses_gdf.iterrows():
    marker = Marker(location=(row.lat, row.lon), draggable=False)
    layer_group.add_layer(marker);


m.add_layer(layer_group)

m

## build a route history 

#### (e.g. all observations ever made for a single route)
#### by iterating over list of shipments and load them all into a dataframe (takes a few minutes)

In [ ]:
def get_route_history(route):
    
    # get the list of shipments for a route
    shipments_for_route_url = f'{api_url}/nyc/{route}'
    shipment_list = requests.get(shipments_for_route_url).json()
    
    # init list to hold results of the individual fetches
    rows=[]

    # iterate over the list of shipments and get each one
    for s in shipment_list['shipments']:
        shipment = requests.get(s['url']).json()
        for bus_dict in shipment['buses']:
            rows.append(bus_dict)
        
    df = pd.DataFrame.from_dict(rows, orient='columns')
    df['passenger_count'] = df['passenger_count'].fillna(0)
    print(f'loaded {len(rows)} buses from {len(shipment_list["shipments"])} shipments for route {route} into DataFrame: "df"')
    
    return df

df = get_route_history(route)
df

In [ ]:
### convert this to a geodataframe
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.lon, df.lat))

In [ ]:
### now make a pydeck map of a random trip?
# https://pydeck.gl/gallery/trips_layer.html